## Equipe Bordeaux

L'équipe est composé d'étudiants de CESI en option **Data Science** organisé de cette manière :
- Elise Beaumatin (Chef de Projet)
- Axel Brosset (Data Scientist)
- Gaylord Girer (Data Scientist)
- Mathieu Musard (Scribe et Misogyne)
- Xavier Labarbe (Modo Discord et Git Master)

[![](https://mermaid.ink/img/pako:eNpVj7uOAjEMRX_Fcs38wBRIDINooFoKRDKFIWYnUh4jJ0EgxL-TXYEErnx9TnF9x1M0jC3-Ck0j7HodoM5CrZxNDB1T8ZRtGKBp5tCpxZUddBJT4jy83H-0VGu6uSgG1la-Wa-2lEfLBbYlkZhPtlJ7ulgW2NCR5MgDztCzeLKmlrr_mRrzyJ41tnU1fKbiskYdHlWlkuPPLZywzVJ4hmUylLm3VN_x2J7JpXqdKBxifOfHE6HJTu0?type=png)](https://mermaid.live/edit#pako:eNpVj7uOAjEMRX_Fcs38wBRIDINooFoKRDKFIWYnUh4jJ0EgxL-TXYEErnx9TnF9x1M0jC3-Ck0j7HodoM5CrZxNDB1T8ZRtGKBp5tCpxZUddBJT4jy83H-0VGu6uSgG1la-Wa-2lEfLBbYlkZhPtlJ7ulgW2NCR5MgDztCzeLKmlrr_mRrzyJ41tnU1fKbiskYdHlWlkuPPLZywzVJ4hmUylLm3VN_x2J7JpXqdKBxifOfHE6HJTu0)

Cette équipe sera mise en compétition contre celle de Pau et celle de Toulouse. Elle vous montrera sa démarche et son travail pour arriver à obtenir l'Intelligence Artificielle la plus performante pour ce projet.

## 1. Importation des modules et des données
Pour créer et faire fonctionner cette IA nous aurons besoin d'importer certains modules ainsi que de la données pour entrainer et valider nos modèles de Deep Learning.
### 1.1 Importation des modules

In [ ]:
import tensorflow as tf
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from zipfile import ZipFile
import shutil
import glob
from termcolor import colored

yellow = '\033[93m'
green = '\033[92m'
red = '\033[91m'
blue = '\033[94m'

Les principaux modules dont nous aurons besoin seront tensorflow pour créer et utiliser notre modèle, matplotlib pour afficher nos résultats et pandas pour formater et utiliser nos données.
### 1.2 Extraction des données
Nos données étant livré sous un format zip nous avons décider d'automatiser leur extraction et leur rangement dans deux catégories **`Photo`** et **`Other`**  qui correspondent à la classification binaire de notre livrable. Si l'image est une photo elle sera rangé dans  **`Photo`** et si l'image est un peinture, un dessin, un plan ou un texte, elle sera rangé dans **`Other`**.
Nous ajoutons un paramètre `allow_extraction` permettant à notre fonction de ne pas s'éxécuter si l'on se souhaite pas et passe la partie extraction des données.

In [ ]:
allow_extraction = False
datasets_path = r"C:\Users\axelb\Desktop\Projet DS\Datasets"
extracted_datasets_path = os.path.join(datasets_path,'Extracted')
new_dataset_filename = "Other"
new_dataset_path = os.path.join(extracted_datasets_path, new_dataset_filename)
datasets_to_regroup = ['Dataset','Painting','Schematics','Sketch','Text']

if allow_extraction:
    try:
        for dataset_filename in os.listdir(datasets_path):

            dataset_path = os.path.join(datasets_path, dataset_filename)
            dataset_zip = ZipFile(dataset_path, 'r')

            if not os.path.exists(extracted_datasets_path):
                os.makedirs(extracted_datasets_path)
                print(f"Extraction folder successfully created at '{extracted_datasets_path}'")

            print(f"Starting the extraction of '{dataset_filename}' at '{extracted_datasets_path}'")
            dataset_zip.extractall(extracted_datasets_path)
            print(f"'{dataset_filename}' was successfully extracted at '{extracted_datasets_path}'")

        dataset_zip.close()
    except:
        pass
    
    if not os.path.exists(new_dataset_path):
        os.makedirs(new_dataset_path)
        print(f"New folder successfully created at '{new_dataset_path}'")
    for dataset in os.listdir(extracted_datasets_path):
        if dataset in datasets_to_regroup:
            dataset_directory = os.path.join(extracted_datasets_path, dataset)
            print(f"Starting the copy of '{dataset_directory}' at '{new_dataset_path}'")
            shutil.copytree(dataset_directory, new_dataset_path, copy_function=shutil.move, dirs_exist_ok=True)
            print(f"'The content of {dataset_directory}' was successfully extracted at '{new_dataset_path}'")
            shutil.rmtree(dataset_directory)

### 1.3 Pré-traitement des données
#### Recherche d'images corrompues et rotation des images
Notre équipe a décidé suite à de premiers essais infructueux d'implémenter une fonction de vérification de fichiers corrompus. Et par la même occasion nous avons décider d'ajouter une fonction pour tourner .

In [ ]:
allow_search_for_corrupted = False
allow_rotation = False

images_width=[]
images_height=[]

if allow_search_for_corrupted:
    img_paths = glob.glob(os.path.join(r"C:\Users\axelb\Desktop\Projet DS\Datasets\Extracted",'*/*.*')) # assuming you point to the directory containing the label folders.

    bad_paths = []

    for image_path in img_paths:
        try:
            img_bytes = tf.io.read_file(image_path)

            decoded_img = tf.io.decode_image(img_bytes)
            img_height = decoded_img.shape[0]
            img_width = decoded_img.shape[1]

            if img_width < img_height and allow_rotation:

                print(f"Rotating the image at {image_path}...")
                decoded_img = tf.image.rot90(decoded_img)
                img_height = decoded_img.shape[0]
                img_width = decoded_img.shape[1]
                
                img = tf.keras.utils.array_to_img(decoded_img)
                rgb_im = img.convert('RGB')
                rgb_im.save(image_path)

            images_width.append(img_width)
            images_height.append(img_height)    
            
        except tf.errors.InvalidArgumentError as e:
          print(f"Found bad path {image_path}...{e}")
          bad_paths.append(image_path)
            
    for bad_path in bad_paths:
        print(f"WARNING : Dropping the file at {bad_path} because it can't be open...")
        os.remove(bad_path)
        
print(f"The median of width is {np.median(images_width)}")
print(f"The median of height is {np.median(images_height)}")

### 1.4 Importation des données
Une fois nos données trier dans des catégories distinctes, les images corrompues retirées et des images retournées ajoutées, nous pouvons importer nos images dans des sets d'images utilisable par **Tensorflow**.

In [ ]:
train_set, test_set = tf.keras.preprocessing.image_dataset_from_directory(
    extracted_datasets_path,
    validation_split=0.2,
    seed=10,
    image_size = (224, 224),
    batch_size=64,
    subset = "both"
)

print(f"\nThe dataset is composed of these classes : {train_set.class_names}")

for images, labels in train_set:
    print(f"\nThe dataset is composed of these classes : {images.shape}")
    print(f"The dataset is composed of these labels : {labels.shape}")
    break

Nous pouvons désormais afficher des images comme par exemple :

In [ ]:
plt.figure(figsize=(8, 8))
for images, labels in train_set.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(train_set.class_names[labels[i]])
        plt.axis("off")
plt.show()